<a href="https://colab.research.google.com/github/msskx/deepLearning/blob/main/softmax%E5%A4%9A%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [12]:
data=pd.read_csv('/content/drive/MyDrive/torch学习笔记/iris.csv')

In [47]:
data.head()

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,1,5.1,3.5,1.4,0.2,setosa
1,2,4.9,3.0,1.4,0.2,setosa
2,3,4.7,3.2,1.3,0.2,setosa
3,4,4.6,3.1,1.5,0.2,setosa
4,5,5.0,3.6,1.4,0.2,setosa


In [57]:
Y=np.array(pd.factorize(data.Species)[0])

In [58]:
X=np.array(data.iloc[:,1:5])

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
train_x,test_x,train_y,test_y=train_test_split(X ,Y,shuffle=True,test_size=0.3)

In [61]:
train_x=torch.from_numpy(train_x).type(torch.float32)
train_y=torch.from_numpy(train_y).type(torch.LongTensor)
test_x=torch.from_numpy(test_x).type(torch.float32)
test_y=torch.from_numpy(test_y).type(torch.LongTensor)

In [62]:
from torch.utils.data import TensorDataset,DataLoader

In [63]:
from sklearn.utils import shuffle
train_ds=TensorDataset(train_x,train_y)
train_dl=DataLoader(train_ds,batch_size=16)

test_ds=TensorDataset(test_x,test_y)
test_dl=DataLoader(test_ds,batch_size=16)

## 模型创建

In [64]:
from torch import nn
import torch.nn.functional as F

In [75]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.liner_1=nn.Linear(4,32)
    self.liner_2=nn.Linear(32,32)
    self.liner_3=nn.Linear(32,1)
  def forward(self,input):
    x=torch.relu(self.liner_1(input))
    x=torch.relu(self.liner_2(x))
    x=torch.sigmoid(self.liner_3(x))
    return x
    

In [76]:
ms=Model()

In [77]:
optim=torch.optim.Adam(ms.parameters(),lr=0.001)

In [81]:
loss_fn=torch.nn.CrossEntropyLoss()

In [82]:
import time

def train(model, optimizer, loss_fn, train_dl, val_dl, epochs=20, device='cuda'):
    '''
    Runs training loop for classification problems. Returns Keras-style
    per-epoch history of loss and accuracy over training and validation data.

    Parameters
    ----------
    model : nn.Module
        Neural network model
    optimizer : torch.optim.Optimizer
        Search space optimizer (e.g. Adam)
    loss_fn :
        Loss function (e.g. nn.CrossEntropyLoss())
    train_dl : 
        Iterable dataloader for training data.
    val_dl :
        Iterable dataloader for validation data.
    epochs : int
        Number of epochs to run
    device : string
        Specifies 'cuda' or 'cpu'

    Returns
    -------
    Dictionary
        Similar to Keras' fit(), the output dictionary contains per-epoch
        history of training loss, training accuracy, validation loss, and
        validation accuracy.
    '''

    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    history = {} # Collects per-epoch loss and acc like Keras' fit().
    history['loss'] = []
    history['val_loss'] = []
    history['acc'] = []
    history['val_acc'] = []

    start_time_sec = time.time()

    for epoch in range(epochs):

        # --- TRAIN AND EVALUATE ON TRAINING SET -----------------------------
        model.train()
        train_loss         = 0.0
        num_train_correct  = 0
        num_train_examples = 0

        for batch in train_dl:

            optimizer.zero_grad()

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            loss = loss_fn(yhat, y)

            loss.backward()
            optimizer.step()

            train_loss         += loss.data.item() * x.size(0)
            num_train_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_train_examples += x.shape[0]

        train_acc   = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_dl.dataset)


        # --- EVALUATE ON VALIDATION SET -------------------------------------
        model.eval()
        val_loss       = 0.0
        num_val_correct  = 0
        num_val_examples = 0

        for batch in val_dl:

            x    = batch[0].to(device)
            y    = batch[1].to(device)
            yhat = model(x)
            loss = loss_fn(yhat, y)

            val_loss         += loss.data.item() * x.size(0)
            num_val_correct  += (torch.max(yhat, 1)[1] == y).sum().item()
            num_val_examples += y.shape[0]

        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_dl.dataset)


        print('Epoch %3d/%3d, train loss: %5.2f, train acc: %5.2f, val loss: %5.2f, val acc: %5.2f' % \
              (epoch+1, epochs, train_loss, train_acc, val_loss, val_acc))

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['acc'].append(train_acc)
        history['val_acc'].append(val_acc)

    # END OF TRAINING LOOP


    end_time_sec       = time.time()
    total_time_sec     = end_time_sec - start_time_sec
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))

    return history

In [83]:
train(ms, optim, loss_fn, train_dl, test_dl, epochs=20, device='cpu')

train() called: model=Model, opt=Adam(lr=0.001000), epochs=20, device=cpu



IndexError: ignored